# Examen: Modulo 1

## 1.0 Limpieza de datos

### 1.1 Detección y remoción de variables poco pobladas. 65% 

In [18]:
from libreria_modulo_1 import analysis, preprocessing, visualization 
import pandas as pd
import numpy as np

In [19]:
datos = pd.read_csv('datos/restaurants.csv')
datos.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [20]:
# se modifico la siguiente funcion para que los "?" sean considerados como datos faltantes
datos = datos.replace('?', np.nan).infer_objects(copy=False)
analysis.completitud_datos(datos)

C:\Users\leono\AppData\Local\Temp\ipykernel_15984\3112654630.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos = datos.replace('?', np.nan).infer_objects(copy=False)


fax               1.0000
url               0.8923
zip               0.5692
country           0.2154
address           0.2077
city              0.1385
state             0.1385
name              0.0000
latitude          0.0000
placeID           0.0000
the_geom_meter    0.0000
longitude         0.0000
alcohol           0.0000
smoking_area      0.0000
dress_code        0.0000
accessibility     0.0000
price             0.0000
Rambience         0.0000
franchise         0.0000
area              0.0000
other_services    0.0000
dtype: float64

In [21]:
df_limpio = preprocessing.delete_missing_values(datos, porcentage=0.65)
print("Forma del DataFrame limpio:", df_limpio.shape) 

Análisis inicial:
- Filas: 130
- Columnas originales: 21
- Porcentaje máximo de NaN permitido por columna: 65.0%
- Máximo de NaN permitidos por columna: 84/130

Análisis por columnas:
- Columnas con más de 65.0% de NaN: 2
- Columnas a eliminar: ['fax', 'url']

Resultados:
- Columnas eliminadas: 2
- Columnas restantes: 19
- Forma final del DataFrame: (130, 19)
Forma del DataFrame limpio: (130, 19)


In [22]:
analysis.completitud_datos(df_limpio)

zip               0.5692
country           0.2154
address           0.2077
state             0.1385
city              0.1385
placeID           0.0000
latitude          0.0000
longitude         0.0000
name              0.0000
the_geom_meter    0.0000
alcohol           0.0000
smoking_area      0.0000
dress_code        0.0000
accessibility     0.0000
price             0.0000
Rambience         0.0000
franchise         0.0000
area              0.0000
other_services    0.0000
dtype: float64

### 1.2  Detección y tratamiento de valores ausentes. 


In [23]:
datos_imputados = preprocessing.impute_missing_values(df_limpio, method='auto')


Valores faltantes antes de imputar:
165 en total
Columna 'address' (categórica): imputada con moda 'Ricardo B. Anaya'
Columna 'city' (categórica): imputada con moda 'San Luis Potosi'
Columna 'state' (categórica): imputada con moda 'SLP'
Columna 'country' (categórica): imputada con moda 'Mexico'
Columna 'zip' (categórica): imputada con moda '78000'

Valores faltantes después de imputar: 0


In [24]:
datos_imputados.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,zip,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,78000,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,78280,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,78000,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,Mexico,78000,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,78000,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none


### 1.3  Detección y remoción de valores extremos.


In [25]:
outliers_iqr = preprocessing.detect_outliers_iqr(datos_imputados, factor=1.5)
outliers_zscore = preprocessing.detect_outliers_zscore(datos_imputados, threshold=3.0)

In [26]:
outliers_iqr

,columna,Q1,Q3,IQR,limite_inferior,limite_superior,num_outliers,porcentaje_outliers
0,placeID,132831.000000,135050.750000,2219.750000,129501.375000,138380.375000,0,0.000000
1,latitude,22.139574,22.160125,0.020551,22.108748,22.190951,46,35.384615
2,longitude,-100.987838,-99.220458,1.767380,-103.638908,-96.569388,0,0.000000


In [27]:
outliers_zscore

,columna,media,std,threshold,num_outliers,porcentaje_outliers
0,placeID,134012.784615,1140.644582,3.0,0,0.0
1,latitude,21.858863,1.496830,3.0,0,0.0
2,longitude,-100.342933,0.860585,3.0,0,0.0
